In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('max_rows', 300)
pd.set_option('max_columns', 300)
import glob
import lightgbm as lgbm
from sklearn.model_selection import KFold

import os
import sys
sys.path.append('..')

from utils import calc_wap, calc_wap2, log_return, realized_volatility, count_unique, calc_mean_importance, calc_model_importance, plot_importance, rmspe, feval_RMSPE
from preprocess import create_all_feature

In [6]:
NB = "007"
feature_path = "../output/feature/nb" + NB
model_path = "../output/model/nb" + NB
if not os.path.isdir(feature_path):
    os.makedirs(feature_path)
if not os.path.isdir(model_path):
    os.makedirs(model_path)


In [7]:
data_dir = '../input/optiver-realized-volatility-prediction/'

In [8]:
df_train, df_test = create_all_feature()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 112 out of 112 | elapsed:  6.4min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.2s finished


In [9]:
df_train.head()

,row_id,target,log_return_realized_volatility,log_return_mean,log_return_std,log_return_sum,log_return2_realized_volatility,log_return2_mean,log_return2_std,log_return2_sum,wap_balance_mean,wap_balance_std,wap_balance_sum,price_spread_mean,price_spread_std,price_spread_sum,bid_spread_mean,bid_spread_std,bid_spread_sum,ask_spread_mean,ask_spread_std,ask_spread_sum,volume_imbalance_mean,volume_imbalance_std,volume_imbalance_sum,total_volume_mean,total_volume_std,total_volume_sum,wap_mean,wap_std,wap_sum,log_return_realized_volatility_450,log_return_mean_450,log_return_std_450,log_return_sum_450,log_return2_realized_volatility_450,log_return2_mean_450,log_return2_std_450,log_return2_sum_450,wap_balance_mean_450,wap_balance_std_450,wap_balance_sum_450,price_spread_mean_450,price_spread_std_450,price_spread_sum_450,bid_spread_mean_450,bid_spread_std_450,bid_spread_sum_450,ask_spread_mean_450,ask_spread_std_450,ask_spread_sum_450,volume_imbalance_mean_450,volume_imbalance_std_450,volume_imbalance_sum_450,total_volume_mean_450,total_volume_std_450,total_volume_sum_450,wap_mean_450,wap_std_450,wap_sum_450,log_return_realized_volatility_300,log_return_mean_300,log_return_std_300,log_return_sum_300,log_return2_realized_volatility_300,log_return2_mean_300,log_return2_std_300,log_return2_sum_300,wap_balance_mean_300,wap_balance_std_300,wap_balance_sum_300,price_spread_mean_300,price_spread_std_300,price_spread_sum_300,bid_spread_mean_300,bid_spread_std_300,bid_spread_sum_300,ask_spread_mean_300,ask_spread_std_300,ask_spread_sum_300,volume_imbalance_mean_300,volume_imbalance_std_300,volume_imbalance_sum_300,total_volume_mean_300,total_volume_std_300,total_volume_sum_300,wap_mean_300,wap_std_300,wap_sum_300,log_return_realized_volatility_150,log_return_mean_150,log_return_std_150,log_return_sum_150,log_return2_realized_volatility_150,log_return2_mean_150,log_return2_std_150,log_return2_sum_150,wap_balance_mean_150,wap_balance_std_150,wap_balance_sum_150,price_spread_mean_150,price_spread_std_150,price_spread_sum_150,bid_spread_mean_150,bid_spread_std_150,bid_spread_sum_150,ask_spread_mean_150,ask_spread_std_150,ask_spread_sum_150,volume_imbalance_mean_150,volume_imbalance_std_150,volume_imbalance_sum_150,total_volume_mean_150,total_volume_std_150,total_volume_sum_150,wap_mean_150,wap_std_150,wap_sum_150,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_log_return_realized_volatility_450,trade_seconds_in_bucket_count_unique_450,trade_size_sum_450,trade_order_count_mean_450,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300,trade_log_return_realized_volatility_150,trade_seconds_in_bucket_count_unique_150,trade_size_sum_150,trade_order_count_mean_150,stock_id,stock_id_target_enc,log_return_realized_volatility_mean_stock,log_return_realized_volatility_std_stock,log_return_realized_volatility_max_stock,log_return_realized_volatility_min_stock,log_return2_realized_volatility_mean_stock,log_return2_realized_volatility_std_stock,log_return2_realized_volatility_max_stock,log_return2_realized_volatility_min_stock,log_return_realized_volatility_450_mean_stock,log_return_realized_volatility_450_std_stock,log_return_realized_volatility_450_max_stock,log_return_realized_volatility_450_min_stock,log_return2_realized_volatility_450_mean_stock,log_return2_realized_volatility_450_std_stock,log_return2_realized_volatility_450_max_stock,log_return2_realized_volatility_450_min_stock,log_return_realized_volatility_300_mean_stock,log_return_realized_volatility_300_std_stock,log_return_realized_volatility_300_max_stock,log_return_realized_volatility_300_min_stock,log_return2_realized_volatility_300_mean_stock,log_return2_realized_volatility_300_std_stock,log_return2_realized_volatility_300_max_stock,log_return2_realized_volatility_300_min_stock,log_return_realized_volatility_150_mean_stock,log_return_realized_volatility_1

In [10]:
df_test.head()

,stock_id,time_id,row_id,log_return_realized_volatility,log_return_mean,log_return_std,log_return_sum,log_return2_realized_volatility,log_return2_mean,log_return2_std,log_return2_sum,wap_balance_mean,wap_balance_std,wap_balance_sum,price_spread_mean,price_spread_std,price_spread_sum,bid_spread_mean,bid_spread_std,bid_spread_sum,ask_spread_mean,ask_spread_std,ask_spread_sum,volume_imbalance_mean,volume_imbalance_std,volume_imbalance_sum,total_volume_mean,total_volume_std,total_volume_sum,wap_mean,wap_std,wap_sum,log_return_realized_volatility_450,log_return_mean_450,log_return_std_450,log_return_sum_450,log_return2_realized_volatility_450,log_return2_mean_450,log_return2_std_450,log_return2_sum_450,wap_balance_mean_450,wap_balance_std_450,wap_balance_sum_450,price_spread_mean_450,price_spread_std_450,price_spread_sum_450,bid_spread_mean_450,bid_spread_std_450,bid_spread_sum_450,ask_spread_mean_450,ask_spread_std_450,ask_spread_sum_450,volume_imbalance_mean_450,volume_imbalance_std_450,volume_imbalance_sum_450,total_volume_mean_450,total_volume_std_450,total_volume_sum_450,wap_mean_450,wap_std_450,wap_sum_450,log_return_realized_volatility_300,log_return_mean_300,log_return_std_300,log_return_sum_300,log_return2_realized_volatility_300,log_return2_mean_300,log_return2_std_300,log_return2_sum_300,wap_balance_mean_300,wap_balance_std_300,wap_balance_sum_300,price_spread_mean_300,price_spread_std_300,price_spread_sum_300,bid_spread_mean_300,bid_spread_std_300,bid_spread_sum_300,ask_spread_mean_300,ask_spread_std_300,ask_spread_sum_300,volume_imbalance_mean_300,volume_imbalance_std_300,volume_imbalance_sum_300,total_volume_mean_300,total_volume_std_300,total_volume_sum_300,wap_mean_300,wap_std_300,wap_sum_300,log_return_realized_volatility_150,log_return_mean_150,log_return_std_150,log_return_sum_150,log_return2_realized_volatility_150,log_return2_mean_150,log_return2_std_150,log_return2_sum_150,wap_balance_mean_150,wap_balance_std_150,wap_balance_sum_150,price_spread_mean_150,price_spread_std_150,price_spread_sum_150,bid_spread_mean_150,bid_spread_std_150,bid_spread_sum_150,ask_spread_mean_150,ask_spread_std_150,ask_spread_sum_150,volume_imbalance_mean_150,volume_imbalance_std_150,volume_imbalance_sum_150,total_volume_mean_150,total_volume_std_150,total_volume_sum_150,wap_mean_150,wap_std_150,wap_sum_150,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_log_return_realized_volatility_450,trade_seconds_in_bucket_count_unique_450,trade_size_sum_450,trade_order_count_mean_450,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300,trade_log_return_realized_volatility_150,trade_seconds_in_bucket_count_unique_150,trade_size_sum_150,trade_order_count_mean_150,stock_id_target_enc,log_return_realized_volatility_mean_stock,log_return_realized_volatility_std_stock,log_return_realized_volatility_max_stock,log_return_realized_volatility_min_stock,log_return2_realized_volatility_mean_stock,log_return2_realized_volatility_std_stock,log_return2_realized_volatility_max_stock,log_return2_realized_volatility_min_stock,log_return_realized_volatility_450_mean_stock,log_return_realized_volatility_450_std_stock,log_return_realized_volatility_450_max_stock,log_return_realized_volatility_450_min_stock,log_return2_realized_volatility_450_mean_stock,log_return2_realized_volatility_450_std_stock,log_return2_realized_volatility_450_max_stock,log_return2_realized_volatility_450_min_stock,log_return_realized_volatility_300_mean_stock,log_return_realized_volatility_300_std_stock,log_return_realized_volatility_300_max_stock,log_return_realized_volatility_300_min_stock,log_return2_realized_volatility_300_mean_stock,log_return2_realized_volatility_300_std_stock,log_return2_realized_volatility_300_max_stock,log_return2_realized_volatility_300_min_stock,log_return_realized_volatility_150_mean_stock,log_return_realized_volatility_

# LightGBM

In [11]:
X = df_train.drop(['row_id','target'],axis=1)
y = df_train['target']

In [12]:
params = {
      "objective": "rmse", 
      "metric": "rmse", 
      "boosting_type": "gbdt",
      'early_stopping_rounds': 30,
      'learning_rate': 0.01,
      'lambda_l1': 1,
      'lambda_l2': 1,
      'feature_fraction': 0.8, # 利用する特徴量の割合
      'bagging_fraction': 0.8,
  }

In [13]:
kf = KFold(n_splits=5, random_state=55, shuffle=True)
oof = pd.DataFrame()
models = []
scores = 0.0

In [14]:
%%time
for fold, (trn_idx, val_idx) in enumerate(kf.split(X, y)):
    print(f'Fold: {fold+1}')
    X_train, y_train = X.loc[trn_idx], y.loc[trn_idx]
    X_valid, y_valid = X.loc[val_idx], y.loc[val_idx]
    
    # RMSPEで最適化を行うため? RMSEの損失関数に1/yi^2の重み付けをすればOK
    # https://www.kaggle.com/c/optiver-realized-volatility-prediction/discussion/250324
    weights = 1/np.square(y_train)
    lgbm_train = lgbm.Dataset(X_train, y_train, weight=weights)
    
    weights = 1/np.square(y_valid)
    lgbm_valid = lgbm.Dataset(X_valid, y_valid, reference=lgbm_train, weight=weights)
    
    model = lgbm.train(params=params,
                  train_set=lgbm_train,
                  valid_sets=[lgbm_train, lgbm_valid],
                  num_boost_round=5000,         
                  feval=feval_RMSPE,
                  verbose_eval=100,
                  categorical_feature = ['stock_id']                
                 )
    
    y_pred = model.predict(X_valid, num_iteration=model.best_iteration)

    RMSPE = round(rmspe(y_true = y_valid, y_pred = y_pred),3)
    print(f'Performance of the　prediction: , RMSPE: {RMSPE}')

    #keep scores and models
    scores += RMSPE / 5
    models.append(model)
    print("*" * 100)

Fold: 1


/home/yoshikawa/.pyenv/versions/3.7.10/envs/kaggle/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/home/yoshikawa/.pyenv/versions/3.7.10/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1706: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['stock_id']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.213437 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 51459
[LightGBM] [Info] Number of data points in the train set: 343145, number of used features: 230


/home/yoshikawa/.pyenv/versions/3.7.10/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/home/yoshikawa/.pyenv/versions/3.7.10/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Info] Start training from score 0.001800
Training until validation scores don't improve for 30 rounds
[100]	training's rmse: 0.000654502	training's RMSPE: 0.30294	valid_1's rmse: 0.000658341	valid_1's RMSPE: 0.30458
[200]	training's rmse: 0.000520594	training's RMSPE: 0.24096	valid_1's rmse: 0.00052849	valid_1's RMSPE: 0.24451
[300]	training's rmse: 0.000487681	training's RMSPE: 0.22572	valid_1's rmse: 0.000497449	valid_1's RMSPE: 0.23015
[400]	training's rmse: 0.000475107	training's RMSPE: 0.2199	valid_1's rmse: 0.000484966	valid_1's RMSPE: 0.22437
[500]	training's rmse: 0.000467363	training's RMSPE: 0.21632	valid_1's rmse: 0.000477561	valid_1's RMSPE: 0.22095
[600]	training's rmse: 0.000461312	training's RMSPE: 0.21352	valid_1's rmse: 0.000472293	valid_1's RMSPE: 0.21851
[700]	training's rmse: 0.000456051	training's RMSPE: 0.21108	valid_1's rmse: 0.000467718	valid_1's RMSPE: 0.21639
[800]	training's rmse: 0.000451644	training's RMSPE: 0.20904	valid_1's rmse: 0.000464117	v

...

[LightGBM] [Info] Start training from score 0.001798
Training until validation scores don't improve for 30 rounds
[100]	training's rmse: 0.000653718	training's RMSPE: 0.30293	valid_1's rmse: 0.000660497	valid_1's RMSPE: 0.30414
[200]	training's rmse: 0.000519716	training's RMSPE: 0.24083	valid_1's rmse: 0.000528378	valid_1's RMSPE: 0.24331
[300]	training's rmse: 0.000487057	training's RMSPE: 0.2257	valid_1's rmse: 0.000496486	valid_1's RMSPE: 0.22862
[400]	training's rmse: 0.000474476	training's RMSPE: 0.21987	valid_1's rmse: 0.000484813	valid_1's RMSPE: 0.22325
[500]	training's rmse: 0.000466689	training's RMSPE: 0.21626	valid_1's rmse: 0.000478109	valid_1's RMSPE: 0.22016
[600]	training's rmse: 0.000460479	training's RMSPE: 0.21338	valid_1's rmse: 0.000473018	valid_1's RMSPE: 0.21781
[700]	training's rmse: 0.000455375	training's RMSPE: 0.21102	valid_1's rmse: 0.000469104	valid_1's RMSPE: 0.21601
[800]	training's rmse: 0.000450813	training's RMSPE: 0.2089	valid_1's rmse: 0.000465861	v

/home/yoshikawa/.pyenv/versions/3.7.10/envs/kaggle/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/home/yoshikawa/.pyenv/versions/3.7.10/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1706: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['stock_id']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.165977 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 51459
[LightGBM] [Info] Number of data points in the train set: 343146, number of used features: 230


/home/yoshikawa/.pyenv/versions/3.7.10/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/home/yoshikawa/.pyenv/versions/3.7.10/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Info] Start training from score 0.001801
Training until validation scores don't improve for 30 rounds
[100]	training's rmse: 0.000655095	training's RMSPE: 0.30304	valid_1's rmse: 0.000652926	valid_1's RMSPE: 0.30275
[200]	training's rmse: 0.000520878	training's RMSPE: 0.24096	valid_1's rmse: 0.00052181	valid_1's RMSPE: 0.24195
[300]	training's rmse: 0.000487942	training's RMSPE: 0.22572	valid_1's rmse: 0.000490401	valid_1's RMSPE: 0.22739
[400]	training's rmse: 0.000475488	training's RMSPE: 0.21996	valid_1's rmse: 0.000478653	valid_1's RMSPE: 0.22194
[500]	training's rmse: 0.000467753	training's RMSPE: 0.21638	valid_1's rmse: 0.000471951	valid_1's RMSPE: 0.21883
[600]	training's rmse: 0.000461496	training's RMSPE: 0.21349	valid_1's rmse: 0.000466761	valid_1's RMSPE: 0.21643
[700]	training's rmse: 0.000456281	training's RMSPE: 0.21107	valid_1's rmse: 0.000462585	valid_1's RMSPE: 0.21449
[800]	training's rmse: 0.000451928	training's RMSPE: 0.20906	valid_1's rmse: 0.000459189	

In [15]:
scores

0.2032

# Model保存

In [16]:
import pickle

for i, model in enumerate(models):
    pickle.dump(model, open(model_path+"/lgbm"+str(i)+".pkl", 'wb'))

# 特徴量保存

In [17]:
pickle.dump(df_train, open(os.path.join(feature_path, "train.pkl"), 'wb'))
pickle.dump(df_test, open(os.path.join(feature_path, "test.pkl"), 'wb'))

In [18]:
X_valid.columns.to_series().to_csv(os.path.join(feature_path, "columns.csv"), index=False)

In [19]:
X_valid.columns

Index(['log_return_realized_volatility', 'log_return_mean', 'log_return_std',
       'log_return_sum', 'log_return2_realized_volatility', 'log_return2_mean',
       'log_return2_std', 'log_return2_sum', 'wap_balance_mean',
       'wap_balance_std',
       ...
       'trade_log_return_realized_volatility_450_max_time',
       'trade_log_return_realized_volatility_450_min_time',
       'trade_log_return_realized_volatility_300_mean_time',
       'trade_log_return_realized_volatility_300_std_time',
       'trade_log_return_realized_volatility_300_max_time',
       'trade_log_return_realized_volatility_300_min_time',
       'trade_log_return_realized_volatility_150_mean_time',
       'trade_log_return_realized_volatility_150_std_time',
       'trade_log_return_realized_volatility_150_max_time',
       'trade_log_return_realized_volatility_150_min_time'],
      dtype='object', length=230)